In [24]:
import pandas as pd
import mysql.connector
import numpy as np
#import altair as alt
#from altair_saver import save # to save the altair graphs
#import selenium # to save the altair graphs
import os
import dropbox
#import knmi_gn_001 # get daily data from knmi
from datetime import date
from datetime import datetime, timedelta
import statistics
import logging


In [1]:
def readsettings():
    # Read generic settings
    # path = os.getcwd()
    # path = "/home/pi/Documents/settings_GN"
    # # path = "/home/gert/pi/settings_GN"
    # TOKENS = path+'/GN_values.txt'
    TOKENS = 'GN_values.txt'
    token_file = open(TOKENS)
    gn_user,gn_pw,gn_host,gn_dbase,gn_drpbx = token_file.read().split('|')
    token_file.close()
    a = {'user': gn_user,
         'pw': gn_pw,
         'host': gn_host,
         'dbase': gn_dbase,
         'drpbox': gn_drpbx}
#     return(gn_user,gn_pw,gn_host,gn_dbase,gn_drpbx)
    return(a)


In [26]:
def SetPOI(periodstring):       
    if periodstring == None:
        periodstring = "VANDAAG"
    if periodstring.upper() == "GISTEREN" :
        poi = {'start': (date.today() - pd.Timedelta(hours=24)).strftime('%Y-%m-%d'), 
               'end': (date.today()).strftime('%Y-%m-%d')}
    elif periodstring.upper() =="VANDAAG" :
        poi = {'start': (date.today()).strftime('%Y-%m-%d') , 
               'end': (date.today()+ pd.Timedelta(hours=24)).strftime('%Y-%m-%d')}
    else :
        poi = {'start': periodstring.split(":")[0] , 
               'end' : periodstring.split(":")[1]}
        for i in poi:
            if poi[i]=="":

                if i=='start' :
                    dummy = datetime.strptime("2020-12-01",'%Y-%m-%d').date()
                    poi[i] = dummy.strftime('%Y-%m-%d')
                else:
                    poi[i] = (date.today() + pd.Timedelta(hours=24)).strftime('%Y-%m-%d')
                    # poi[i] = (date.today()+ pd.Timedelta(hours=24)).strftime('%Y-%m-%d')
    print("POI is : ",poi)

    return(poi)


In [11]:
def getSQL(poi,dbase_str,gn_settings,var_str):
    sql_select_Query = 'select * from ' + dbase_str + " WHERE time >='" + str(poi['start']) + "'" + "AND time <'" + str(poi['end']) + "'"

    cnx = mysql.connector.connect(user=gn_settings['user'], password=gn_settings['pw'],
                              host=gn_settings['host'],database=gn_settings['dbase'])
    cursor = cnx.cursor()
    cursor.execute(sql_select_Query)
    mysql_df = pd.DataFrame(cursor.fetchall())
    cnx.close()

    #  TO DO : Error if no  values available
    if mysql_df.empty :
        mysql_df = None
    else: 
        mysql_df.columns = ['datetime', "counter"]
        mysql_df['datetime'] = pd.to_datetime(mysql_df['datetime'])
        mysql_df['variable'] = var_str
        mysql_df = mysql_df.set_index('datetime')
        mysql_df['dy'] = mysql_df.index.date

        # erence for each timestamp wrt previous one
        #mysql_df['delta']=mysql_df['counter'].diff()

    return mysql_df

In [12]:
def RemoveIncompleteDays(df_in,pct):
        # in case domotica system does not update the values in the database, datapoints are missing
        # Remove the days when less than pct of datapoints is available
        # als today er bij zit, mag het aantal punten minder zijn. 
        # Een dag mag meetellen als er minimaal pct aan datapunten beschikbaar is.
        # Voor vandaag moet het aantal punten op pct van de verstreken tijd zijn, 
        # voor andere dagen pct van 1440 (60*24) datapunten
    
    df = df_in.resample("d").count()
    df1_ = df.reset_index().groupby('dy')['datetime'].max().to_frame()
    df1_.columns = ["max"]
    df2_ = df.reset_index().groupby('dy')['datetime'].min().to_frame()
    df2_.columns = ["min"]
    df_ = pd.concat([df1_,df2_],axis=1)
    df_['delta'] = pd.to_timedelta(df_['max'] - df_['min']).dt.total_seconds()/60
    df = pd.concat([df,df_],axis=1)

    df['ratio2'] = df['counter'] /  df['delta']
    df['ratio'] = df['counter'] /  (60*24)
    df.loc[df.index == date.today().strftime('%Y-%m-%d'),'ratio']  = df.loc[df.index == date.today().strftime('%Y-%m-%d'),'ratio2']
    df['var'] = df_in.iloc[0]['variable']
    
    df_excluded = df[df ['ratio'] < pct/100]
    excluded = df_excluded.index
    
    if len(excluded)>0 :
        message =" Dates that do NOT meet treshold"
        df_excluded = df_excluded[['dy','ratio','var']] 
        print(df_excluded)
    else: 
        message ="All Datapoints do meet treshold"

    print(message)
    return df_excluded

In [13]:
def RemoveReadingErrors(df):
    # Deze functie moet per dag geprocessed worden
    # Gas readings blijken niet altijd even goed te zijn, af en toe zelfs een factor 10 te hoog. 
    # remove the outliers
    # interpolate the data, assuming the outliers do not occur for a long period.

    try: 
        data_std = statistics.pstdev(df['counter'])
        data_median = df['counter'].median()   
        upper_limit = data_median + 40
        if df['counter'].max() > upper_limit:
            anomalies = df.loc[(df['counter'] > upper_limit)].index
            print("Anomalies removed :",anomalies)

            if len(anomalies)>0:
                df.loc[(df['counter']> upper_limit),['counter']]= np.nan
                df['counter'] = df['counter'].fillna(method="ffill")
        else:
            None
            # print("No extreme values detected",np.unique(df['dy']))
    except:
        print("No Data")
    return df

In [14]:
def RemoveReadingErrors_r1(df):
    
    """ Gas readings blijken niet altijd even goed te zijn, af en toe zelfs een factor 10 te hoog. 
    remove the outliers
    interpolate the data, assuming the outliers do not occur for a long period.
    
    dus op t1 een misreading van 100 keer de counter met op dat moment een heel hoog verbruik
    vervolgens loopt die misreading een aantal minuten door, met verbruik van 0 of realistisch verbruik
    dan valt de waarde weer terug naar de juiste counter, dus een heel groot negatief getal. 
    echter stel dat de P1 meter een tijdje niet geregistreed heeft dan wordt een ongewijzigde stand opgeslagen
    in de database en op het moment dat P1 meter weer actief wordt zal er een relatief grote stap in couter 
    optreden, deze moet er niet (of niet direct) uitgefilterd worden.
    
    Oplossings richting: de counter mag enkel 1 richting op tellen, en zal nooit lager wroden.
    als er een negatieve stap zichtbaar is, dan is op een eerder moment een te hoge waarde geregistreerd 
    of komt het ook voor dat eerst een negatieve stap gemaakt wordt die later gecorrigeerd wordt?
    """
    
    # check per dag op outliers
    days = df.index.strftime("%Y-%m-%d").unique()
 
    for d in days:
        dummy = df[df['dy'].astype(str)==d]
        dagmax = dummy['counter'].tail(1).squeeze()
        dagmin = dummy['counter'].head(1).squeeze()

        logging.info('dy : %s. dagmin:%s , dagmax:%s',str(d),dagmin,dagmax)  
       
        if dummy[dummy['counter']>dagmax].shape[0]>0:
            logging.info('Too high values detected !')
            mask = (df['dy'].astype(str)==d)&(df['counter']>dagmax)
            df.loc[mask,'counter'] = np.nan # dit klopt niet want teveel data wordt overschreven waar enkel de dagdata veranderd zou moeten worden
        elif dummy[dummy['counter']<dagmin].shape[0]>0:
            logging.info('Too low values detected !')
            mask = (df['dy'].astype(str)==d)&(df['counter']<dagmin)
            df.loc[mask,'counter'] = np.nan 
            
            
        df['counter'] =  df['counter'].fillna(method="ffill")
        
    return df

In [15]:
# df_g = df_g.groupby(['dy']).apply(lambda x : RemoveReadingErrors(x))

In [16]:
def RemoveSensorOutages(df):
    # when sensor is out of order, the same value is recorded every minute.
    # Remove the days when number of identical recordings exceed a manually set treshold
    #
    # Difficulty : sommige dagen blijft de teller gewoon uren onveranderd en is er niets mis.
    varname = df.iloc[0]['variable']
    anomalies = []
    try: 
        df_counts = df.groupby(['dy','counter']).count().reset_index()  # count nr of identical readings
        df_counts= df_counts.groupby('dy')['variable'].max().to_frame() #max nr of identical readings per day
        # if nr datapoints exceeds treshold exclude the day
        SensorOutages = df_counts[df_counts['variable']>500].index

        if len(SensorOutages)>0:
            print("Days with too many sensor outages of ", varname ," : ", SensorOutages)
            anomalies = SensorOutages.to_list()

    except:
        print("Error in processing Sensor outages")
    return anomalies



In [1]:
def retrieveSQLdata_day(dbase_str,poi,gn_settings):
    poistart = poi
    poiend =  datetime.strptime(poi, '%Y-%m-%d') + timedelta(days=1)
    poiend = poiend.strftime("%Y-%m-%d")
    sql_select_Query = 'select * from ' + dbase_str + " WHERE time >='" + poistart + "'" + "AND time <'" + poiend + "'"
    
    cnx = mysql.connector.connect(user=gn_settings['user'], password=gn_settings['pw'],
                              host=gn_settings['host'],database=gn_settings['dbase'])
    cursor = cnx.cursor()
    cursor.execute(sql_select_Query)
    mysql_df = pd.DataFrame(cursor.fetchall())
    cnx.close()

    #  TO DO : Error if no  values available
    if mysql_df.empty :
        mysql_df = None
    else: 
        mysql_df.columns = ['datetime', "counter"]
        mysql_df['datetime'] = pd.to_datetime(mysql_df['datetime'])
        mysql_df = mysql_df.set_index('datetime')
        mysql_df['dy'] = mysql_df.index.date

        # erence for each timestamp wrt previous one
        #mysql_df['delta']=mysql_df['counter'].diff()

    return mysql_df

In [17]:
# class RetrieveRawData:
    
#     def __init__(self, name_str, dbase_str, periodstring):
#             self.varname = name_str
#             self.dbase_str = 'select * from ' + dbase_str
#             self.DataUnavailable =[]
            
#             # initalisation of period of interest
            
#             if periodstring == None:
#                 periodstring = "VANDAAG"
#             if periodstring.upper() == "GISTEREN" :
#                 poi = {'start': (date.today() - pd.Timedelta(hours=24)).strftime('%Y-%m-%d'), 
#                        'end': (date.today()).strftime('%Y-%m-%d')}
#             elif periodstring.upper() =="VANDAAG" :
#                 poi = {'start': (date.today()).strftime('%Y-%m-%d') , 
#                        'end': (date.today()+ pd.Timedelta(hours=24)).strftime('%Y-%m-%d')}
#             else :
#                 poi = {'start': periodstring.split(":")[0] , 
#                        'end' : periodstring.split(":")[1]}
#                 for i in poi:
#                     if poi[i]=="":

#                         if i=='start' :
#                             dummy = datetime.strptime("2020-12-01",'%Y-%m-%d').date()
#                             poi[i] = dummy.strftime('%Y-%m-%d')
#                         else:
#                             poi[i] = (date.today()).strftime('%Y-%m-%d')
#             print("POI is : ",poi)

            
#             self.poi_start = poi['start']
#             self.poi_end = poi['end']
#             #self.data = []            
            
#     def getSQL(self,gn_settings):
#         sql_select_Query = self.dbase_str + " WHERE time >='" + str(self.poi_start) + "'" + "AND time <'" + str(self.poi_end) + "'"
        
#         cnx = mysql.connector.connect(user=gn_settings['user'], password=gn_settings['pw'],
#                                   host=gn_settings['host'],database=gn_settings['dbase'])
#         cursor = cnx.cursor()
#         cursor.execute(sql_select_Query)
#         mysql_df = pd.DataFrame(cursor.fetchall())
#         cnx.close()
            
#         #  TO DO : Error if no  values available
#         if mysql_df.empty :
#             self.data = None
#         else: 
#             mysql_df.columns = ['datetime', "counter"]
#             mysql_df['datetime'] = pd.to_datetime(mysql_df['datetime'])
#             mysql_df['variable'] = self.varname
#             mysql_df = mysql_df.set_index('datetime')

#             # erence for each timestamp wrt previous one
#             #mysql_df['delta']=mysql_df['counter'].diff()
#             self.data = mysql_df

    
#     def RemoveIncompleteDays(self,pct):
#         # in case domotica system does not update the values in the database, datapoints are missing
#         # Remove the days when less than pct of datapoints is available
#         # als today er bij zit, mag het aantal punten minder zijn. 
#         # Een dag mag meetellen als er minimaal pct aan datapunten beschikbaar is.
#         # Voor vandaag moet het aantal punten op pct van de verstreken tijd zijn, 
#         # voor andere dagen pct van 1440 (60*24) datapunten
        
#         self.data['dy'] = self.data.index.date

#         df = self.data.resample("d").count()
#         df1_ = self.data.reset_index().groupby('dy')['datetime'].max().to_frame()
#         df1_.columns = ["max"]
#         df2_ = self.data.reset_index().groupby('dy')['datetime'].min().to_frame()
#         df2_.columns = ["min"]
#         df_ = pd.concat([df1_,df2_],axis=1)
#         df_['delta'] = pd.to_timedelta(df_['max'] - df_['min']).dt.total_seconds()/60
#         df = pd.concat([df,df_],axis=1)
           
#         df['ratio2'] = df['counter'] /  df['delta']
#         df['ratio'] = df['counter'] /  (60*24)
#         df.loc[df.index == date.today().strftime('%Y-%m-%d'),'ratio']  = df.loc[df.index == date.today().strftime('%Y-%m-%d'),'ratio2']

#         excluded = df[df ['ratio'] < pct/100].index
#         self.incompletedays = excluded
#         self.DataUnavailable = self.DataUnavailable + excluded.to_list()
        
#         if len(excluded)>0 :
#             message =" Nr of datapoints does NOT meet treshold"
#             self.data = self.data[~self.data['dy'].isin(excluded)]
#         else: 
#             message =" Nr of datapoints does meet treshold"
        
#         print(self.varname," : ",message)
        

#         #self.data['diff'] = self.data['counter'].diff().fillna(0)
        
    
    
#     def RemoveReadingErrors(self):
#         # Gas readings blijken niet altijd even goed te zijn, af en toe zelfs een factor 10 te hoog. 
#         # remove the outliers
#         # interpolate the data, assuming the outliers occur for long period.
        
#         try: 
#             data_std = statistics.pstdev(self.data['counter'])
#             data_median = self.data['counter'].median()   
#             upper_limit = data_median + 40
#             if self.data['counter'].max() > upper_limit:
            
# #             if self.data['counter'].max()-self.data['counter'].min() > data_median:
# #                 anomaly_cut_off = data_std * 3
# #                 lower_limit  = data_median - anomaly_cut_off 
# #                 print("ll:",lower_limit)

# #                 upper_limit = data_median + anomaly_cut_off
# #                 print("ul:",upper_limit)

# # #                 self.anomalies = self.data.loc[(self.data['counter']<lower_limit) | (self.data['counter'] > upper_limit)]
#                 self.anomalies = self.data.loc[(self.data['counter']>upper_limit)]
# #                 self.DataUnavailable = self.DataUnavailable + self.anomalies.to_list()
#                 print("Anomalies removed :",self.anomalies)
                
#                 if len(self.anomalies)>0:
#                     print("Nr datapoints addressed as outlier :",len(self.anomalies))
                

# #                     self.data.loc[(self.data['counter']<lower_limit) | (self.data['counter'] > upper_limit),['counter']]= np.nan
#                     self.data.loc[(self.data['counter']> upper_limit),['counter']]= np.nan                            
                                                
#                     self.data['counter'] = self.data['counter'].fillna(method="ffill")
# #                     print("Nr datapoints after removal :",len(self.data.index))
#             else:
#                 print(self.varname," No extreme values detected")
#                 self.anomalies=[]
#         except:
#             print("No Data")

#     def RemoveSensorOutages(self):
#         # when sensor is out of order, the same value is recorded every minute.
#         # Remove the days when number of identical recordings exceed a manually set treshold
        
#         try: 
#             df_counts = self.data.groupby(['dy','counter']).count().reset_index()
#             df_counts= df_counts.groupby('dy')['variable'].max().to_frame() #max nr of identical readings per day
#             # if nr datapoints exceeds treshold exclude the day
#             self.SensorOutages = df_counts[df_counts['variable']>300].index
            
#             if len(self.SensorOutages)>0:
#                 print("Days with too many sensor outages of ", self.varname ," : ", self.SensorOutages)
#                 self.data = self.data[~self.data['dy'].isin(self.SensorOutages)]
#                 self.DataUnavailable = self.DataUnavailable + self.SensorOutages.to_list()
    
#         except:
#             print("Error in processing Sensor outages")
# #         return(self.data[~self.data['dy'].isin(self.SensorOutages)])
#         return(self.data[self.data['dy'].isin(self.SensorOutages)])
         
        
        

In [18]:
def RemoveDatesFromOther(df,errordates):
    
#     df = df[~df['dy'].astype(str).isin(errordates)]
    df = df[~df['dy'].isin(errordates)]
    return(df)

In [19]:
def calcdiff(x):
    # x is a DataFrame of group values
#     x['diff'] = x['counter'].diff().fillna(0)
    x['diff'] = x['counter'] - x['counter'].shift(1)
    x['diff'] = x['diff'].fillna(0)
    return x

In [20]:
def retrieveGasandTemps(poi,pct):
    gn_req = readsettings()
        
    df_g = getSQL(poi, 'item0054',gn_req,"gas")
    df_t = getSQL(poi,'item0057',gn_req, 'tapwater')
    df_c = getSQL(poi,'item0056',gn_req, 'CVout')
    df = [df_c,df_t,df_g]
    
#     df_g = RetrieveRawData('gas','item0054',poi)
#     df_t = RetrieveRawData('tapwater','item0057',poi)
#     df_c = RetrieveRawData('CVout','item0056',poi)

#     df_g.getSQL(gn_req)
#     df_t.getSQL(gn_req)
#     df_c.getSQL(gn_req)
    
    df_excl = []
    for i in df:
        df_excl.append(RemoveIncompleteDays(i,pct))
    df_excl = pd.DataFrame(pd.concat(df_excl)).reset_index().pivot(index="index", columns="var", values="ratio")
    print(df_excl)
#     treshold = pct # percent of day should at least be available,
#     df_g.RemoveIncompleteDays(treshold)
#     df_t.RemoveIncompleteDays(treshold)
#     df_c.RemoveIncompleteDays(treshold)
    
#     # gas shows anomalies
#     df_g.RemoveReadingErrors()
    df_g = df_g.groupby(['dy']).apply(lambda x : RemoveReadingErrors(x))    
    
#     df_t.RemoveSensorOutages()
#     df_g.RemoveSensorOutages()
#     df_c.RemoveSensorOutages()
    
#     # merge into one dataframe
#     # first remove the dates in the other datasets
#     DaysUnavailable = list(set(df_c.DataUnavailable) | set(df_t.DataUnavailable) | set(df_g.DataUnavailable) )

#     df_c.data = RemoveDatesFromOther(df_c.data,DaysUnavailable)
#     df_t.data = RemoveDatesFromOther(df_t.data,DaysUnavailable)
#     df_g.data = RemoveDatesFromOther(df_g.data,DaysUnavailable)
    
        
    df = [df_c,df_t,df_g]
#     df = pd.concat(df)

    df = pd.concat(df)
    df['dy'] = pd.to_datetime(df['dy'])
    df = df[~df['dy'].isin(df_excl.index)]
    
    df = df.reset_index()
    df = df.groupby(['variable','dy']).apply(calcdiff)
    
# #     df['diff'] = df.groupby(['variable','dy'])['counter'].diff().fillna(0)
    
    if not df.empty : 
        df1 = df.reset_index().pivot(index='datetime', columns='variable', values='counter')
        df2 =df.reset_index().pivot(index='datetime', columns='variable', values='diff')
        df = pd.concat([df1, df2], axis=1)
        df.columns = ["abs_CVout","abs_gas","abs_tapwater","diff_CVout","diff_gas","diff_tapwater"]
    
    return (df)

In [27]:
if __name__ == "__main__":
#     poi = "gisteren"
#     poi = "2020-12-31:"
    poi = "vandaag"
    poi = SetPOI(poi)
#     poi = "vandaag"
#     poi = "2021-03-07:2021-03-13"
#     poi = "2020-06-12:2020-06-13"
    pct = 90
    df = retrieveGasandTemps(poi,pct)
    
        


POI is :  {'start': '2022-06-18', 'end': '2022-06-19'}
All Datapoints do meet treshold
All Datapoints do meet treshold
All Datapoints do meet treshold
Empty DataFrame
Columns: []
Index: []


In [22]:
# df2

In [ ]:
# if df_g.data['counter'].max()-df_g.data['counter'].min() > data_median:
# anomaly_cut_off = data_std * 3
# lower_limit  = data_median - anomaly_cut_off 
# print("ll:",lower_limit)

# upper_limit = data_median + anomaly_cut_off
# print("ul:",upper_limit)

# self.anomalies = df_g.data.loc[(df_g.data['counter']<lower_limit) | (df_g.data['counter'] > upper_limit)]
# #                 df_g.dataUnavailable = df_g.dataUnavailable + self.anomalies.to_list()
# print("Anomalies removed :",self.anomalies)

# if len(self.anomalies)>0:
# print("Nr datapoints addressed as outlier :",len(self.anomalies))

# #                     df_g.data = df_g.data.loc[(df_g.data['counter']>=lower_limit) & (df_g.data['counter'] <= upper_limit)]

# df_g.data.loc[(df_g.data['counter']<lower_limit) | (df_g.data['counter'] > upper_limit),['counter']]= np.nan
# df_g.data['counter'] = df_g.data['counter'].fillna(method="ffill")
# #                     print("Nr datapoints after removal :",len(df_g.data.index))

In [ ]:
# df_g.data[['counter']].median()

In [ ]:
# df_g = RetrieveRawData('gas','item0054',poi)
# # df_g.getSQL(gn_req)
    
# # treshold = pct # percent of day should at least be available,
# # df_g.RemoveIncompleteDays(treshold)
# # # gas shows anomalies

In [ ]:
# # pd.to_datetime(df.index.date).dt.date.unique()
# d = {'dates': df.index.strftime('%Y-%m-%d').unique()}
# datesdf = pd.DataFrame(data=d)
# datesdf.groupby([pd.to_datetime(datesdf['dates']).dt.year, pd.to_datetime(datesdf['dates']).dt.month]).agg(['count'])

In [ ]:
# gn_req = readsettings()
# df_g = RetrieveRawData('gas','item0054',poi)
# df_g.getSQL(gn_req)

In [ ]:
# df_g.data['dy']=df_g.data.index.date

In [ ]:
# df_g.data.head()

In [57]:
# df_g.data.dtypes

In [ ]:
# import altair as alt

# palette = alt.Scale(domain=['gas'],
#                     range=["#1696d2"])   
  
# source = df_g.data.reset_index().sample(n=5000)
# source['datetime'] = pd.to_datetime(source['datetime'])
# chart1a = alt.Chart(source,title="Gas").mark_line(
#         strokeWidth=1, point=True
#     ).encode(
#         x = alt.X('datetime:T',axis=alt.Axis(title="Tijdstip")),
#         y = alt.Y("counter:Q",axis=alt.Axis(title="Temperatuur [dC]")),
#         color = alt.Color("variable",scale = palette)
#     ).properties(width=800,height=100)

In [ ]:
# chart1a

In [ ]:
# dummy = df_g.data.groupby(['dy']).apply(calcdiff)
# dummy = dummy.groupby("dy").sum("diff")

In [ ]:
# palette = alt.Scale(domain=['gas'],
#                     range=["#1696d2"])   
   
# source = dummy.reset_index()
# source['dy'] =pd.to_datetime(source['dy'])
# chart1a = alt.Chart(source,title="Gas").mark_bar().encode(
#         x = alt.X('dy:T',axis=alt.Axis(title="Tijdstip")),
#         y = alt.Y('diff:Q',axis=alt.Axis(title="Verbruik [dC]"))
#     ).properties(width=800,height=100)
# chart1a

In [ ]:
# dummy.reset_index().dtypes

In [ ]:
# # pd.to_datetime(df.index.date).dt.date.unique()
# d = {'dates': df_g.data.index.strftime('%Y-%m-%d').unique()}
# datesdf = pd.DataFrame(data=d)
# datesdf.groupby([pd.to_datetime(datesdf['dates']).dt.year, pd.to_datetime(datesdf['dates']).dt.month]).agg(['count'])